<a href="https://colab.research.google.com/github/ThrishaJawahar/Falcon-9-Launch-Prediction/blob/main/InteractiveDashboardwithPlotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:

import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px


spacex_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(spacex_url)


max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


app = dash.Dash(__name__)


app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    html.Br(),

    # TASK 1: Add a dropdown list to enable Launch Site selection

    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
        ],
        value='ALL',
        placeholder="Select a Launch Site",
        style={'width': '50%', 'padding': '3px'}
    ),
    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),


    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=100,
        marks={0: '0', 100: '100', 500: '500', 1000: '1000'},
        value=[min_payload, max_payload],
        tooltip={'always_visible': True}
    ),
    html.Br(),

    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# TASK 2: Callback for pie chart based on selected site
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':

        df_filtered = spacex_df.groupby('class').size().reset_index(name='count')
        labels = ['Failure', 'Success']
        values = df_filtered['count']
    else:

        df_filtered = spacex_df[spacex_df['Launch Site'] == selected_site]
        df_filtered = df_filtered.groupby('class').size().reset_index(name='count')
        labels = ['Failure', 'Success']
        values = df_filtered['count']


    fig = px.pie(values=values, names=labels, title=f"Launch Success for {selected_site}")
    return fig

# TASK 4: Callback for scatter chart based on payload range and selected site
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    df_filtered = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) &
                            (spacex_df['Payload Mass (kg)'] <= high)]

    if selected_site != 'ALL':

        df_filtered = df_filtered[df_filtered['Launch Site'] == selected_site]


    fig = px.scatter(df_filtered, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                     title=f"Launch Success vs. Payload for {selected_site}",
                     labels={'class': 'Launch Success', 'Payload Mass (kg)': 'Payload Mass (kg)'})
    return fig


if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>